## Noisy communication plot development


Began with the heatmaps of average final polarization over 50 trials, each run to 10k iterations.

In [2]:
import h5py
import numpy as np
import os
import pandas as pd

from glob import glob

nedir = 'data/noise_experiment_all/'

g = glob(os.path.join(nedir, '*'))
print(len(g))

726
